In [1]:
import os
import glob
import shutil
import json
import numpy as np
from PIL import Image
from tqdm.auto import tqdm as tqdm
import numpy as np
import pandas as pd
from itertools import product

np.random.seed(42)

In [5]:
method = 'Harmonized_Tires_Artificial'
root_dir = os.path.join('..','data', 'data_aug', 'GAN')

data_dir = os.path.join(root_dir, method)
annot_path = os.path.join(root_dir, method, 'labels')

out_dir = data_dir
images_path = os.path.join(root_dir, method)

json_name = 'coco_format_%s_fold%s_%s.json'

In [ ]:
# separating images by video folders
images = glob.glob(images_path+'/*.png')
for image in tqdm(images):
    root = image.split(method)[0]
    root = os.path.join(root, method)
    
    video = image.split(method)[-1][1:18]
    if not os.path.isdir(os.path.join(root, video)):
        os.makedirs(os.path.join(root, video))
    shutil.copyfile(image, os.path.join(root, video, image[-14:]))    

In [3]:
def get_img_size(datapath):
    img_pil = Image.open(datapath)
    width, height = img_pil.size
    return width, height


def get_groundtruth(img_path, annotation_folder):
    annot_path = None
    if annotation_folder is not None:
        # look for annotation file
        annot_path = _find_annot_file(img_path, annotation_folder)

    if annot_path is not None:
        frame_number = _get_frame_number(img_path)
        annotation = AnnotationImage(frame_number, annot_path)
        boxes, labels = annotation.get_bboxes_labels()

        return boxes, labels

    return [], []

In [4]:
k = 2 #fold
file_set = '../train_sets_kfold.xls'
df_sets = pd.read_excel(file_set, sheet_name=f'train_sets_k{k}')
df_sets

,Video,bottle,bucket,pool,puddle,tire,water_tank,local,set_1,set_2,set_3,set_4,test
0,20180905_rectfied_DJI_0010,34,6,4,0,21,0,Campo Gremio,1.0,1.0,1.0,NaN,NaN
1,20180919_rectfied_DJI_0018,9,10,5,0,24,0,Atras Bloco I,1.0,NaN,NaN,NaN,NaN
2,20180919_rectfied_DJI_0019,34,10,5,0,25,0,Atras Bloco I,1.0,NaN,NaN,NaN,NaN
3,20180919_rectfied_DJI_0021,0,11,0,0,0,204,Acima bloco I (video longo),1.0,1.0,1.0,1.0,NaN
4,20181022_rectfied_DJI_0025,19,6,8,0,22,0,aeroclube,NaN,NaN,NaN,NaN,1.0
5,20181022_rectfied_DJI_0027,14,6,6,0,13,0,aeroclube,NaN,NaN,NaN,NaN,1.0
6,20181022_rectfied_DJI_0031,24,13,12,0,31,0,Atras FAU (wild),NaN,NaN,NaN,NaN,1.0
7,20181029_rectfied_DJI_0033,17,27,20,0,20,5,CCMN,NaN,NaN,NaN,NaN,1.0
8,20181029_rectfied_DJI_0034,5,26,24,0,30,5,CCMN,NaN,NaN,NaN,NaN,1.0
9,20181029_rectfied_DJI_0038,43,32,18,6,40,0,frente FAU,1.0,1.0,1.0,1.0,NaN


In [12]:
include_reals = True
subsets = ['set_1', 'set_2', 'set_3', 'set_4']

sets = product([method], subsets)

category_dict = {
    #'__background__': 0,
    'tire': 0,
    }

for method, subset in tqdm(sets):
    print(f'Starting {method, subset}')
    
    videos_set = df_sets[df_sets[subset] == True]['Video'].tolist()
    videos_set = [video[9:] for video in videos_set]
        
    ann_dict = {}
    images = []
    annotations = []
        
    img_id = 0
    ann_id = 0
    cat_id = 1
    
#     filenames = glob.glob(os.path.join(data_dir, data_set,'*.png'))
    
    for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir))):
        if len(filenames) == 0:
            continue
    
        video_name = os.path.split(dirpath)[-1]        
        if video_name not in videos_set:
            continue
        
        for filename in tqdm(filenames):
            if filename.lower().endswith(('.png')):               
                image = {}
                
                img_name_short = os.path.join(video_name, filename)
                img_name = os.path.join(dirpath, filename)
                
                    
                width, height = get_img_size(img_name)
                
                image['file_name'] = img_name_short
                image['width'] = width
                image['height'] = height
                image['id'] = img_id
                                
                #read .txt in images folder
                if include_reals:                    
                    anno_filepath = os.path.join(root_dir, "labels_reals", 
                                                 video_name.replace('rectfied', 'rectified'), 
                                           filename.replace('.png','.txt'))
                    
                    if os.path.isfile(anno_filepath):
                        num_real_in_img = 0
                        
                        with open(anno_filepath, 'r') as marks:
                            for mark in marks.readlines():                                                          
                                rect = mark.replace('\n','').split('\t')                                  
                                
                                if len(rect) <=1:
                                    continue
                                    
                                objid,rcx,rcy,rw,rh = map(float,rect)
                                x0 = int((rcx-rw/2)*3840) #hardcoded pq veio assim da annot
                                y0 = int((rcy-rh/2)*height)
                                w = int(rw*width)
                                h = int(rh*height)

                                ann = {}
                                ann['id'] = ann_id
                                ann['image_id'] = img_id
                                ann['category_id'] = 0
                                ann['iscrowd'] = 0
                                ann['area'] = w*h
                                ann['bbox'] = (x0,y0,w,h)

                                annotations.append(ann)
                                ann_id += 1    
                                num_real_in_img += 1      
                                
                #read .txt in labels folder                                
                anno_filepath = os.path.join(root_dir, "labels_fake", video_name + 
                                   '_'+filename.replace('.png','.txt'))

                with open(anno_filepath, 'r') as marks:

                    for mark in marks.readlines():                            
                        rect = mark.replace('\n','').split('\t')

                        if len(rect) <=1:
                            continue

                        objid,rcx,rcy,rw,rh = map(float,rect)
                        x0 = int((rcx-rw/2)*3840) #hardcoded pq veio assim da annot
                        y0 = int((rcy-rh/2)*height)
                        w = int(rw*width)
                        h = int(rh*height)                            

                        ann = {}
                        ann['id'] = ann_id
                        ann['image_id'] = img_id
                        ann['category_id'] = 0
                        ann['iscrowd'] = 0
                        ann['area'] = w*h
                        ann['bbox'] = (x0,y0,w,h)

                        annotations.append(ann)
                        ann_id += 1
                        
                images.append(image)
                img_id += 1

            
    ann_dict['images'] = images
    categories = [{"id": category_dict[name], "name": name} for name in category_dict]
    ann_dict['categories'] = categories
    ann_dict['annotations'] = annotations
    print("Num categories: %s" % len(categories))
    print("Num images: %s" % len(images))
    print("Num annotations: %s" % len(annotations))
    
    
    with open(os.path.join(out_dir, json_name % (method, k, subset)), 'w') as outfile:
        outfile.write(json.dumps(ann_dict))

Starting ('lum_blurblend', 'set_1')




Num categories: 1
Num images: 2902
Num annotations: 498
Starting ('lum_blurblend', 'set_2')




Num categories: 1
Num images: 2826
Num annotations: 410
Starting ('lum_blurblend', 'set_3')




Num categories: 1
Num images: 2511
Num annotations: 326
Starting ('lum_blurblend', 'set_4')




Num categories: 1
Num images: 2008
Num annotations: 254

